# Generating the input-output function $P(g\mid R, c)$ for varying repressor copy number $R$.

In [1]:
import pickle
import os
import glob
import datetime

# Our numerical workhorses
import numpy as np
from sympy import mpmath
import scipy.optimize
import scipy.special
import scipy.integrate
import pandas as pd
import itertools

# Import libraries to parallelize processes
from joblib import Parallel, delayed

# Import the utils for this project
import chann_cap_utils as chann_cap

# Pre-computing analytical distributions of gene expession.

Since the computation of the mRNA and protein steady-state probability distributions are computationally expensive, we can pre-compute the distribution for different repressor copy number and save the results as a lookup table to compute any desired quantity out of these distributions including the channel capacity and the variability in gene expression due to the stochasticity of the allosteric molecules.

This notebook achieves the simple task of computing the mRNA and protein distribution for different repressor copy numbers saving the result into csv files that we can read with `numpy`.

The matrices are arranged such that each row's index is given by the number of repressors and each column index indicates either the mRNA or protein count.

## Pre-computing the mRNA distribution

Let's start by saving the distribution for mRNA molecules.

In [ ]:
# Define the parameters
k0 = 2.7E-3 # Used by Jones and Brewster

# The MWC parameters come from the global fit to the O2 data
mRNA_params = dict(ka=0.199, ki=0.00064, omega=np.exp(-4.5), 
                   k0=k0, gamma=0.00284, r_gamma=15.7)

In [ ]:
# Define the mRNA copy numbers to evaluate
# It is break up in blocks to run the process in parallel
mRNA_grid = np.reshape(np.arange(0, 50), [-1, 10])

# define the array of repressor copy numbers to evaluate the function in
R_array = np.arange(0, 1001)

kon_array = [chann_cap.kon_fn(-17, mRNA_params['k0']),
             chann_cap.kon_fn(-15.3, mRNA_params['k0']),
             chann_cap.kon_fn(-13.9, mRNA_params['k0']),
             chann_cap.kon_fn(-9.7, mRNA_params['k0'])]
kon_operators = ['Oid', 'O1', 'O2', 'O3']

compute_matrix = True
if compute_matrix:
    for j, kon in enumerate(kon_array):
        print('operator : ' + kon_operators[j])
        # Set the value for the kon
        mRNA_params['kon'] = kon
        # Initialize transition matrix
        QmR = np.zeros([mRNA_grid.size, len(R_array)])
        for i, r in enumerate(R_array):
            if r%100==0:
                print('repressors : {:d}'.format(r))
            mRNA_params['rep'] = r * 1.66
            # -- Parallel computation of distribution -- #
            lnm_list = list()
            # loop through the concentrations
            # define a function to run in parallel the computation
            def lnm_parallel(m):
                lnm = chann_cap.log_p_m_mid_C(C=0, mRNA=m, **mRNA_params)
                return lnm
            lnm_list.append(Parallel(n_jobs=7)(delayed(lnm_parallel)(m) \
                                               for m in mRNA_grid))
            # -- Building and cleaning the transition matrix -- #
            for k, lnm in enumerate(lnm_list):
                # Initialize the matrix of zeros where the normalized
                # distribution will live
                p_norm = np.zeros_like(lnm)
                p = np.exp(lnm)
                # Compute the cumulative sum of the protein copy number
                p_sum = np.cumsum(np.sum(p, axis=1))
                # Find the first block that is already normalized given 
                # the tolerance value
                norm_idx = np.where((p_sum <= 1 + 1E-5) & \
                                    (p_sum >= 1 - 1E-5))[0][-1]
                # add all the probability values of these blocks to our matrix
                p_norm[0:norm_idx, :] = p[0:norm_idx, :]
            QmR[:, i] = p_norm.ravel()
            # Check that all distributions for each concentration are normalized
        np.savetxt('../../tmp/QmR_' + kon_operators[j] +\
                   '_0_1000_literature_param.csv', QmR, delimiter=",")

### Pre-computing the protien distribution

In [2]:
# Protein parameters
k0 = 2.7E-3 # From Jones & Brewster
prot_params = dict(ka=141.52, ki=0.56061, epsilon=4.5,
                   kon=chann_cap.kon_fn(-9.7, k0),
                   k0=k0,
                   gamma_m=0.00284, r_gamma_m=15.7,
                   gamma_p=0.000277, r_gamma_p=100)

In [ ]:
# Define the protein blocks to evaluate in parallel
# Break into blocks to compute the distributions in parallel
prot_grid = np.reshape(np.arange(0, 4000), [-1, 50])

# define the array of repressor copy numbers to evaluate the function in
R_array = np.arange(0, 1050)

# Setting the kon parameter based on k0 and the binding energies form stat. mech.
kon_array = [chann_cap.kon_fn(-13.9, prot_params['k0']),
             chann_cap.kon_fn(-15.3, prot_params['k0']),
             chann_cap.kon_fn(-9.7, prot_params['k0']),
             chann_cap.kon_fn(-17, prot_params['k0'])]
kon_operators = ['O2', 'O1', 'O3', 'Oid']
kon_dict = dict(zip(kon_operators, kon_array))

compute_matrix = True
if compute_matrix:
    for kon, op in enumerate(kon_operators):
        print('operator : ' + op)
        # Set the value for the kon
        prot_params['kon'] = kon_dict[op]
        # Define filename
        file = '../../data/csv_protein_dist/lnp_' + op + '_DJ_RB.csv'
        # Loop through repressor copy numbers
        for i, r in enumerate(R_array):
            if r%50==0:
                print('repressors : {:d}'.format(r))
            prot_params['rep'] = r * 1.66
            # -- Parallel computation of distribution -- #
            # define a function to run in parallel the computation
            def lnp_parallel(p):
                lnp = chann_cap.log_p_p_mid_C(C=0, protein=p, **prot_params)
                df = pd.DataFrame([r] * len(p), index=p, columns=['repressor'])
                df.loc[:, 'protein'] = pd.Series(p, index=df.index)
                df.loc[:, 'lnp'] = lnp
                
                # if file does not exist write header 
                if not os.path.isfile(file): 
                    df.to_csv(file) 
                else: # else it exists so append without writing the header
                    df.to_csv(file, mode='a', header=False)
            Parallel(n_jobs=40)(delayed(lnp_parallel)(p) for p in prot_grid)

operator : O2
repressors : 0
